In [2]:
import pymongo

# database 연결
def connect_database(db):
    connect_to = pymongo.MongoClient("mongodb://localhost:27017")
    mdb = connect_to[db]
    return mdb

# collection 연결
def use(db, collection):
    return db[collection]

def insert(collection, data_list):
    collection.insert_many(data_list)

def find(collection, options = {}):
    searched = collection.find(options)
    return searched

def update(collection, options = {}):
    collection.update_many(options)

def delete(collection, options = {}):
    collection.delete_many(options)

In [3]:
db = connect_database("trip")

In [4]:
test = use(db, "test")

In [5]:
import os
import csv
import pandas as pd

# 관광데이터 포털에서 가져온 지역별 모든 데이터를 몽고db에 저장하는 함수
def test1(path, collection):
    db = connect_database("trip")
    col_name = use(db, collection)
    os.chdir(path)
    filelist = os.listdir(path)
    for file in filelist:
        df = pd.read_csv(file, encoding = "CP949")
        header = df.columns.values.tolist()
        c_file = open(file, encoding = "CP949")
        f_read = csv.DictReader(c_file)
        data = []
        for each in f_read:
            row = {}
            for field in header:
                row[field] = each[field]
            data.append(row)
        insert(col_name, data)

In [6]:
test1(r"C:\Users\삼성\Desktop\재성\한이음 챗봇\데이터\서울특별시", "seoul")

In [7]:
seoul = use(db, "seoul")
cur = find(seoul)

In [17]:
# 몽고 db에 구분 없이 저장된 지역별 모든 데이터를
# csv 파일 별로 리스트에 저장하는 함수
def test2(collection, region):
    db = connect_database("trip")
    col_name = use(db, collection)
    cur = find(col_name)
    data_list = []
    key_names = []
    temp = []
    merged_data = []
    for doc in cur:
        doc.update(지역 = region)
        data_list.append(doc)
    for i in range(len(data_list)):
        key_names.append(data_list[i].keys())
    for key in key_names:
        if key not in temp:
            temp.append(key)
    for j in range(len(temp)):
        tmp = []
        for data in data_list:
            if data.keys() == temp[j]:
                tmp.append(data)
        merged_data.append(tmp)
    return merged_data

In [19]:
seoul_merge = test2("seoul", "서울")
seoul_merge

[[{'_id': ObjectId('62f4a4c397340f4de392785c'),
   '업종 중분류명': '항공운송',
   '소비비율': '167.2',
   '전년도 소비액': '1.04099466391E11',
   '소비액': '2.78144683348E11',
   '지역': '서울'}],
 [{'_id': ObjectId('62f4a4c397340f4de392785d'),
   '시군구명': '서울특별시',
   '평균 체류시간': '177.0',
   '평균 숙박일수': '2.0',
   '체류유형': '휴식형',
   '지역': '서울'}],
 [{'_id': ObjectId('62f4a4c397340f4de392785e'),
   '연령대': '60~69세',
   '성별': '여성',
   '증가율': '18.9',
   '지역': '서울'}],
 [{'_id': ObjectId('62f4a4c397340f4de392785f'),
   '유입지역명': '경기도 ',
   '유출지역명': ' 서울특별시',
   '유입유출 비율': '87.1',
   '유입/유출 구분 코드 (1:유입 / 2:유출)': '1',
   '지역': '서울'},
  {'_id': ObjectId('62f4a4c397340f4de3927860'),
   '유입지역명': '인천광역시 ',
   '유출지역명': ' 서울특별시',
   '유입유출 비율': '9.7',
   '유입/유출 구분 코드 (1:유입 / 2:유출)': '1',
   '지역': '서울'},
  {'_id': ObjectId('62f4a4c397340f4de3927861'),
   '유입지역명': '강원도 ',
   '유출지역명': ' 서울특별시',
   '유입유출 비율': '1.1',
   '유입/유출 구분 코드 (1:유입 / 2:유출)': '1',
   '지역': '서울'},
  {'_id': ObjectId('62f4a4c397340f4de3927862'),
   '유입지역명': '충청남도 ',


In [20]:
#데이터프레임 리스트로 불러오는 함수
def test3(data):
    df_list = []
    for i in range(len(data)):
        df_list.append(pd.DataFrame(data[i]))
    return df_list

In [21]:
df_list = test3(seoul_merge)
df_list[2]

,_id,연령대,성별,증가율,지역
0,62f4a4c397340f4de392785e,60~69세,여성,18.9,서울
